# A*, Heuristics, and the Fifteen Puzzle

In this project, we demonstrate abstracting a problem into a search problem, implementing the A* search algorithm (still the most common way of doing pathfinding in video games), and experimenting with the effects of using different heuristics for the search.

The goal of a fifteen puzzle is to get all fifteen tiles in order from left to right, top to bottom, like so:

1 2 3 4

5 6 7 8

9 10 11 12

13 14 15 -

The only legal moves are to move a tile adjacent to the blank into the blank space, making the tile’s previous space blank. Thus the maximum number of neighbors is 4, but the number of neighbors could be as small as 2 if the blank is in a corner.

**1) Is a count of number of tiles out of place an admissible heuristic if the blank is not counted as a tile? What if the blank is counted as a tile? In each case, if the heuristic is admissible, explain how you know, and if it is not, give an example that shows the heuristic is inadmissible.**


In the first scenario, where the blank space is not counted, the heuristic is admissible. This is because the number of tiles out of place is the number of tile that need to be moved. The empty space will get where it needs to be because if it is not where it needs to be, then at least one other piece will also be out of place. In the second scenario, it is inadmissible because you are overestimating the cost it takes to reach the goal. Take, for example, a simple one-move left example. If the 15 and blank tiles need to be switched, the heuristic would show that there are 2 out of place tiles as opposed to the one. 

In [ ]:
"""Use A* to solve fifteen puzzle instances.

The "main" of this code is solve_and_print, at the end.  We'll try two different
heuristics, counting tiles out of place and summing Manhattan distance from
the destination over all tiles (the better heuristic)."""

import sys
import copy
import numpy as np
from queue import PriorityQueue
import math

PUZZLE_WIDTH = 4
BLANK = 0  # Integer comparison tends to be faster than string comparison

def read_puzzle_string(puzzle_string):
    """Read a NumberPuzzle from string representation; space-delimited, blank is "-".

    Args:
      puzzle_string (string):  string representation of the puzzle

    Returns:
      A NumberPuzzle
    """
    new_puzzle = NumberPuzzle()
    row = 0
    for line in puzzle_string.splitlines():
        tokens = line.split()
        for i in range(PUZZLE_WIDTH):
            if tokens[i] == '-':
                new_puzzle.tiles[row][i] = BLANK
                new_puzzle.blank_r = row
                new_puzzle.blank_c = i
            else:
                try:
                    new_puzzle.tiles[row][i] = int(tokens[i])
                except ValueError:
                    sys.exit("Found unexpected non-integer for tile value")
        row += 1
    return new_puzzle

class NumberPuzzle(object):
    """ Class containing the state of the puzzle, as well as A* bookkeeping info.

    Attributes:
        tiles (numpy array): 2D array of ints for tiles.
        blank_r (int):  Row of the blank, for easy identification of neighbors
        blank_c (int):  Column of blank, same reason
        parent (NumberPuzzle):  Reference to previous puzzle, for backtracking later
        dist_from_start (int):  Steps taken from start of puzzle to here
        key (int or float):  Key for priority queue to determine which puzzle is next
    """

    def __init__(self):
        """ Just return zeros for everything and fill in the tile array later"""
        self.tiles = np.zeros((PUZZLE_WIDTH, PUZZLE_WIDTH))
        self.blank_r = 0
        self.blank_c = 0
        # This next field is for our convenience when generating a solution
        # -- remember which puzzle was the move before
        self.parent = None
        self.dist_from_start = 0
        self.key = 0

    def __str__(self):
        """This is the Python equivalent of Java's toString()."""
        out = ""
        for i in range(PUZZLE_WIDTH):
            for j in range(PUZZLE_WIDTH):
                if j > 0:
                    out += " "
                if self.tiles[i][j] == BLANK:
                    out += "-"
                else:
                    out += str(int(self.tiles[i][j]))
            out += "\n"
        return out

    def copy(self):
        """Copy the puzzle and update the parent field.
        
        In A* search, we generally want to copy instead of destructively alter,
        since we're not backtracking so much as jumping around the search tree.
        Also, if A and B are numpy arrays, "A = B" only passes a reference to B.
        We'll also use this to tell the child we're its parent."""
        child = NumberPuzzle()
        child.tiles = np.copy(self.tiles)
        child.blank_r = self.blank_r
        child.blank_c = self.blank_c
        child.dist_from_start = self.dist_from_start
        child.parent = self
        return child

    def __eq__(self, other):
        """Governs behavior of ==.
        
        Overrides == for this object so that we can compare by tile arrangement
        instead of reference.  This is going to be pretty common, so we'll skip
        a type check on "other" for a modest speed increase"""
        return np.array_equal(self.tiles, other.tiles)

    def __hash__(self):
        """Generate a code for hash-based data structures.
        
        Hash function necessary for inclusion in a set -- unique "name"
        for this object -- we'll just hash the bytes of the 2D array"""
        return hash(bytes(self.tiles))

    def __lt__(self, obj):
        """Governs behavior of <, and more importantly, the priority queue.
        
        Override less-than so that we can put these in a priority queue
        with no problem.  We don't want to recompute the heuristic here,
        though -- that would be too slow to do it every time we need to
        reorganize the priority queue"""
        return self.key < obj.key

    def total_h(self, better_h):
        """A* cost:  admissible heuristic plus cost-so-far.

        Args:
            better_h (boolean):  True for Manhattan distance, false for counting tiles.
          
        Returns:
            A number representing the heuristic value (int or float)
        """
        return self.dist_from_start + self.heuristic(better_h)

    def move(self, tile_row, tile_column):
        """Move from the row, column coordinates given into the blank.

        Also very common, so we will also skip checks for legality to improve speed.

        Args:
            tile_row (int):  Row of the tile to move.
            tile_column (int):  Column of the tile to move.
        """

        self.tiles[self.blank_r][self.blank_c] = self.tiles[tile_row][tile_column]
        self.tiles[tile_row][tile_column] = BLANK
        self.blank_r = tile_row
        self.blank_c = tile_column
        self.dist_from_start += 1

    def legal_moves(self):
        """Return a list of NumberPuzzle states that could result from one move.

        Return a list of NumberPuzzle states that could result from one move
        on the present board.  Use this to keep the order in which
        moves are evaluated the same as our solution, thus matching the
        HackerRank solution as well.  (Also notice we're still in the
        methods of NumberPuzzle, hence the lack of arguments.)

        Returns:
            List of NumberPuzzles.
        """
        legal = []
        if self.blank_r > 0:
            down_result = self.copy()
            down_result.move(self.blank_r-1, self.blank_c)
            legal.append(down_result)
        if self.blank_c > 0:
            right_result = self.copy()
            right_result.move(self.blank_r, self.blank_c-1)
            legal.append(right_result)
        if self.blank_r < PUZZLE_WIDTH - 1:
            up_result = self.copy()
            up_result.move(self.blank_r+1, self.blank_c)
            legal.append(up_result)
        if self.blank_c < PUZZLE_WIDTH - 1:
            left_result = self.copy()
            left_result.move(self.blank_r, self.blank_c+1)
            legal.append(left_result)
        return legal

    def solve(self, better_h):
        """Return a list of puzzle states from this state to solved.

        Args:
            better_h (boolean):  True if Manhattan heuristic, false if tile counting

        Returns:
            path (list of NumberPuzzle or None) - path from start state to finish state
            explored - total number of nodes pulled from the priority queue
        """
        # Initialize open and closed list then add starting node on the open list.
        closed_list = set()
        open_list = PriorityQueue()
        open_list.put(self)

        # Total number of nodes pulled from the priority q.
        explored = 0

        # While open list is not empty.
        while open_list._qsize() != 0:
          explored += 1

          # Pull the current state and see it if requires more work.
          state = open_list._get()
          # Yes.
          if state.solved():
            goal_state = [state]
            parent = state.parent
            while parent is not None:
              goal_state.insert(0, parent)
              parent = parent.parent
            return goal_state, explored
          # No.
          # If a node with the same position as the successor is in the closed
          # list, skip the successor.
          q = state.__hash__()
          if q in closed_list:
            continue

          # Otherwise, add node to the open list.
          # Push q onto the closed list.
          for node in state.legal_moves():
            node.key = node.heuristic(better_h) + node.dist_from_start
            open_list.put(node)
            node.parent = state
          closed_list.add(q)

        return None, explored

    def solved(self):
        """"Return True iff all tiles in order and blank in bottom right."""
        should_be = 1
        for i in range(PUZZLE_WIDTH):
            for j in range(PUZZLE_WIDTH):
                if self.tiles[i][j] != should_be:
                    return False
                should_be = (should_be + 1) % (PUZZLE_WIDTH ** 2)
        return True

    def heuristic(self, better_h):
        """Wrapper for the two heuristic functions.

        Args:
            better_h (boolean):  True if Manhattan heuristic, false if tile counting

        Returns:
            Value of the cost-to-go heuristic (int or float)
        """
        if better_h:
            return self.manhattan_heuristic()
        return self.tile_mismatch_heuristic()

    def tile_mismatch_heuristic(self):
        """Returns count of tiles out of place.
        
        Can't count the blank or it's inadmissible."""
        mismatch_count = 0
        # Completed. This was largely based on the solved() that returns a boolean.
        # Just adapted to show all the reasons it is not solved. 
        should_be = 1
        for ii in range(PUZZLE_WIDTH):
            for jj in range(PUZZLE_WIDTH):
                # Ignores the 0 tile in this line.
                if self.tiles[ii][jj] != should_be and self.tiles[ii][jj] != 0:
                    mismatch_count += 1
                should_be = (should_be + 1) % (PUZZLE_WIDTH ** 2)
        return mismatch_count

    def manhattan_heuristic(self):
        """Returns total Manhattan (city block) distance from destination over all tiles.

        Again, shouldn't count blank; it gets where it's going for free."""
        total_manhattan = 0
        should_be = 1
        x_offset = y_offset = 0
        for ii in range(PUZZLE_WIDTH):
            for jj in range(PUZZLE_WIDTH):
                # Ignores the 0 tile in this line.
                if self.tiles[ii][jj] != should_be and self.tiles[ii][jj] != 0:
                  y_offset = abs(math.ceil(self.tiles[ii][jj] // 4) - ii)
                  x_offset = abs((self.tiles[ii][jj] % 4) - jj)
                  total_manhattan += x_offset + y_offset
                should_be = (should_be + 1) % (PUZZLE_WIDTH ** 2)
        return total_manhattan

    def path_to_here(self):
        """Returns list of NumberPuzzles giving the move sequence to get here.
        
        Retraces steps to this node through the parent fields."""
        path = []
        current = self
        while not current is None:
            path.insert(0, current)  # push
            current = current.parent
        return path

def print_steps(path):
    """ Print every puzzle in the path.

    Args:
        path (list of NumberPuzzle): list of puzzle states from start to finish
    """
    if path is None:
        print("No path found")
    else:
        print("{} steps".format(len(path)-1))
        for state in path:
            print(state)


def solve_and_print(puzzle_string : str, better_h : bool) -> None:
  """ "Main" - prints series of moves necessary to solve puzzle.

  Args:
    puzzle_string (string):  The puzzle to solve.
    better_h (boolean):  True if Manhattan distance heuristic, false if tile count
  """
  my_puzzle = read_puzzle_string(puzzle_string)
  solution_steps, explored = my_puzzle.solve(better_h) 
  print("{} nodes explored".format(explored))
  print_steps(solution_steps)

In solve(), we implement A*, using a heuristic of “number of tiles in the wrong place” as the optimistic estimate of moves to go. To do this, we make use of two important data structures:

• The queue of puzzle states to explore should be a PriorityQueue, already imported for you at the top. The __lt__() function for NumberPuzzle objects has already been overridden so that it compares the key field to decide what goes first, but that field is currently never initialized.

• Use a set() to efficiently implement a "closed list" of states that have already been explored. Sets are hash tables, and the hashing behavior has already been implemented to work in an acceptable way.

solve() should return a list of NumberPuzzles that show the states from the beginning to the end, as well as an integer count of the number of nodes explored (that is, pulled from the front of the priority queue).  The latter is to help you debug and help us grade, although there is some "wiggle room" for reasonable differences in implementation.



In [ ]:
zero_moves = """1 2 3 4
5 6 7 8
9 10 11 12
13 14 15 -"""

one_move = """1 2 3 4
5 6 7 8
9 10 11 12
13 14 - 15"""

six_moves = """1 2 3 4
5 10 6 8
- 9 7 12
13 14 11 15"""

sixteen_moves = """10 2 4 8
1 5 3 -
9 7 6 12
13 14 11 15"""

forty_moves = """4 3 - 11
2 1 6 8
13 9 7 15
10 14 12 5"""



In [ ]:
solve_and_print(zero_moves, False)

1 nodes explored
0 steps
1 2 3 4
5 6 7 8
9 10 11 12
13 14 15 -



In [ ]:
solve_and_print(one_move, False)

2 nodes explored
1 steps
1 2 3 4
5 6 7 8
9 10 11 12
13 14 - 15

1 2 3 4
5 6 7 8
9 10 11 12
13 14 15 -



In [ ]:
solve_and_print(six_moves, False)

7 nodes explored
6 steps
1 2 3 4
5 10 6 8
- 9 7 12
13 14 11 15

1 2 3 4
5 10 6 8
9 - 7 12
13 14 11 15

1 2 3 4
5 - 6 8
9 10 7 12
13 14 11 15

1 2 3 4
5 6 - 8
9 10 7 12
13 14 11 15

1 2 3 4
5 6 7 8
9 10 - 12
13 14 11 15

1 2 3 4
5 6 7 8
9 10 11 12
13 14 - 15

1 2 3 4
5 6 7 8
9 10 11 12
13 14 15 -



**3) Now we time our implementation on sixteen_moves**, using the handy Google Colab syntax demonstrated here.

In [ ]:
%time solve_and_print(sixteen_moves, False)

398 nodes explored
16 steps
10 2 4 8
1 5 3 -
9 7 6 12
13 14 11 15

10 2 4 -
1 5 3 8
9 7 6 12
13 14 11 15

10 2 - 4
1 5 3 8
9 7 6 12
13 14 11 15

10 - 2 4
1 5 3 8
9 7 6 12
13 14 11 15

- 10 2 4
1 5 3 8
9 7 6 12
13 14 11 15

1 10 2 4
- 5 3 8
9 7 6 12
13 14 11 15

1 10 2 4
5 - 3 8
9 7 6 12
13 14 11 15

1 - 2 4
5 10 3 8
9 7 6 12
13 14 11 15

1 2 - 4
5 10 3 8
9 7 6 12
13 14 11 15

1 2 3 4
5 10 - 8
9 7 6 12
13 14 11 15

1 2 3 4
5 10 6 8
9 7 - 12
13 14 11 15

1 2 3 4
5 10 6 8
9 - 7 12
13 14 11 15

1 2 3 4
5 - 6 8
9 10 7 12
13 14 11 15

1 2 3 4
5 6 - 8
9 10 7 12
13 14 11 15

1 2 3 4
5 6 7 8
9 10 - 12
13 14 11 15

1 2 3 4
5 6 7 8
9 10 11 12
13 14 - 15

1 2 3 4
5 6 7 8
9 10 11 12
13 14 15 -

CPU times: user 39.6 ms, sys: 1.99 ms, total: 41.6 ms
Wall time: 44 ms


**4) The Manhattan distance of a tile from its final location is the sum of the difference in rows and the difference in columns. If the blank does not count as a tile, does the sum of Manhattan distances from their final locations over all tiles act as an admissible heuristic? What if the blank does count as a tile? In each case, if the heuristic is admissible, explain how you know, and if it is not, give an example that shows the heuristic is inadmissible.**

Like in question one, we don't count the space because it gets where it is going for free. when moving all other tiles and not considering the work required to get 0 back, the work for 0 to finish in the right spot gets accomplished, therefore, to include it in the heuristic would be over estimating the work and would make it inadmissable. If you consider the test case with 6 moves you can see how it messes up the heuristic based on where the empty space is. If you ignore it, however, you get the otpimal solution.

Now, we implement Manhattan distance as a new heuristic. We keep our old heuristic, but have the code use the old heuristic if the better_h argument is False, and use the new heuristic if better_h is True.  Now we will time the new code.

In [ ]:
%time solve_and_print(sixteen_moves, True)

70 nodes explored
16 steps
10 2 4 8
1 5 3 -
9 7 6 12
13 14 11 15

10 2 4 -
1 5 3 8
9 7 6 12
13 14 11 15

10 2 - 4
1 5 3 8
9 7 6 12
13 14 11 15

10 - 2 4
1 5 3 8
9 7 6 12
13 14 11 15

- 10 2 4
1 5 3 8
9 7 6 12
13 14 11 15

1 10 2 4
- 5 3 8
9 7 6 12
13 14 11 15

1 10 2 4
5 - 3 8
9 7 6 12
13 14 11 15

1 - 2 4
5 10 3 8
9 7 6 12
13 14 11 15

1 2 - 4
5 10 3 8
9 7 6 12
13 14 11 15

1 2 3 4
5 10 - 8
9 7 6 12
13 14 11 15

1 2 3 4
5 10 6 8
9 7 - 12
13 14 11 15

1 2 3 4
5 10 6 8
9 - 7 12
13 14 11 15

1 2 3 4
5 - 6 8
9 10 7 12
13 14 11 15

1 2 3 4
5 6 - 8
9 10 7 12
13 14 11 15

1 2 3 4
5 6 7 8
9 10 - 12
13 14 11 15

1 2 3 4
5 6 7 8
9 10 11 12
13 14 - 15

1 2 3 4
5 6 7 8
9 10 11 12
13 14 15 -

CPU times: user 18.5 ms, sys: 3.05 ms, total: 21.6 ms
Wall time: 23.5 ms


In [ ]:
%time solve_and_print(forty_moves, True)

KeyboardInterrupt: ignored

**7) Suppose you decide to try out Euclidean distance, $\sqrt{r^2 + c^2}$ where r and c are the row and column differences, as a heuristic.  It runs faster than tiles displaced, but slower than Manhattan distance.  Why?  (Assume it's not the slowness of square root operations, or anything like that.)**

The manhattan distance was faster than the number of displaced tiles because it more acurately represents the amount of work to be done by how far away the misplaced tiles actually are. This allows the heuristic to choose its moves better. Similarly, euclidian also represents the amount of work better than number of tiles misplaced, but due to the restraints of the game, misrepresents the work based on the moves allowed. This is why manhattan distance is faster. While manhattan distance will be larger than the euclidian distance, it will be more accurate and will allow for better choices to be made.

**8) Suppose a bug caused you to calculate the Manhattan distance incorrectly, so that it only returned the number of rows away for each tile, ignoring columns.  Is this heuristic still going to return an optimal solution every time?  Why or why not?**

It is possible that certain scenarios would produce an optimal solution, but definitely not every time. The reason for this is that the amount of work for each tile is not being properly represented, so any attempts to prioritize will be done with bad information. This will lead to some sub-optimal solutions.

**9) In some fifteen-puzzle implementations, you can slide not just one tile, but all tiles to one side of the blank into the blank space.  (For example, if the bottom row were - 13 14 15, one move could cause 13 14 15 -.) Are the tiles displaced and Manhattan distance heuristics admissible in that case?  If so, explain how you know.  If not, provide a heuristic that is admissible and can return at least 4 different values, and explain how you know it is admissible.  (Heuristics should not return values less than 0.  Hint:  Consider simple arithmetic to modify the existing heuristics.)**

This new rule change would make the manhattan distance inadmissable. This is because it will be overestimating the amount of work that needs to be done by not taking into account this new rule. A newer heuristic could consider blocks of ordered tiles in the right row and instead of counting them separately, produce values of work based on what would happen if they moved as a singular unit. We would know that this would always be the right choice because the blocks have to stay next to each other in these cases. This would only work if they were in the right row already. The values returned could be 0, 1, 2, for moving left to right, and then tiles moving up and down or tiles not currently ordered in their row could be asessed the typical manhattan way. This would be admissible because instead of counting 3 for the block of [- 13 14 15], it would count 1, which is accurate according to the new rules.

A* is one of the most successful algorithms in the history of AI, a champ at what it does (as long as you can come up with a good heuristic), and is probably the most important AI technology for a modern game AI programmer to understand (because of all the pathfinding that goes on). It's a classic technique for a reason!